In [1]:

import pandas as pd
from urllib.parse import quote
from sqlalchemy import create_engine
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random

def engine_create():
    user = 'root'
    password = '3xlasql'
    host = 'J8E203.p.ssafy.io'
    port = 3306
    database = 'music'

    db_uri = f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4'
    engine = create_engine(db_uri, echo=True)

    return engine.connect()


def refresh_df():
    conn = engine_create()
    ten_emotion = pd.read_sql_table('song_10emotion', con=conn, index_col=0)
    t3_emotion_keysen = pd.read_sql_table('song_emotion_keyword', con=conn, index_col=0)
    print('[song_10emtion, song_emotion_keyword] data load')
    return ten_emotion, t3_emotion_keysen

In [2]:
ten, t3 = refresh_df()

2023-04-04 13:42:11,236 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-04-04 13:42:11,238 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-04 13:42:11,262 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-04-04 13:42:11,263 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-04 13:42:11,278 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-04-04 13:42:11,279 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-04 13:42:11,308 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-04 13:42:11,308 INFO sqlalchemy.engine.Engine DESCRIBE `music`.`song_10emotion`
2023-04-04 13:42:11,310 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-04 13:42:11,380 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `song_10emotion`
2023-04-04 13:42:11,381 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-04 13:42:11,395 INFO sqlalchemy.engine.Engine SELECT song_10emotion.id, song_10emotion.song_id, song_10emotion.anger, song_10emotion.malice, song_10emotion.sad, song_10emotion.des

In [ ]:
def recommend_cosine(df, input_emotion):
    
    input_array = np.array([[int(i) for i in input_emotion]])

    cons = cosine_similarity(df.iloc[:, 2:12].values, input_array)
    
    print(cons)
    
    recommend_id_list = sorted(enumerate(cons), reverse=True, key=lambda x: x[1])
    
    recommend_id_list = [recommend_id_list[:4]]
    
    song_id_list = []
    for idx in recommend_id_list:
        song_id_list.append(df.iloc[idx[0], 1])
        
    return song_id_list

In [34]:
cons = cosine_similarity(ten.iloc[:20, 2:12].values, np.array([[236,   447,  2406,  -159,   805,  -757,   779,  2115, -2863, -2795]]))

In [52]:
recommend_id_list = sorted(enumerate(cons), key= lambda x: x[1], reverse=True)[:2]

for i in recommend_id_list:
    print(ten.iloc[i[0], 1])

52441
52156


In [68]:
input_emotion = [236,   447,  2406,  -159,   805,  -757,   779,  2115, -2863, -2795]

input_rank = sorted(enumerate(input_emotion), key=lambda x:x[1], reverse=True)

top_bottom = input_rank[0:2] + input_rank[8:]

input_rank = [i[0] for i in top_bottom]

input_array = np.array([[input_emotion[i] for i in input_rank]])

cosine_similarity(ten.iloc[:5, input_rank].values, input_array)

array([[-0.74919237],
       [-0.02546029],
       [-0.63126632],
       [-0.64375024],
       [ 0.19246463]])

In [43]:
df = ten
input_emotion = [0.236,   0.447,  2.406,  -0.159,   0.805,  -0.757,   0.779,  2.115, -2.863, -2.795]


input_rank = sorted(enumerate(input_emotion), key=lambda x:x[1], reverse=True)

top_3 = input_rank[0:3]
bottom_2 = input_rank[8:]

input_t3_rank = [i[0] for i in top_3]
input_b2_rank = [i[0] for i in bottom_2]

input_t3_array = np.array([[int(input_emotion[i]*1000) for i in input_t3_rank]])
input_b2_array = np.array([[int(input_emotion[i]*1000) for i in input_b2_rank]])

cons_t = cosine_similarity(df.iloc[:, input_t3_rank].values, input_t3_array)
cons_b = cosine_similarity(df.iloc[:, input_b2_rank].values, input_b2_array)

cons = cons_t + cons_b/5

recommend_id_list = sorted(enumerate(cons), reverse=True, key=lambda x: x[1])

recommend_id_list = recommend_id_list[:1]

song_id_list = []
for idx in recommend_id_list:
    song_id_list.append(df.iloc[idx[0], 1])
    
song_id_list

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 9134 while Y.shape[1] == 1

In [38]:
int(input_emotion[input_b1_rank]*1000)

-2863

In [42]:
np.array([df.iloc[:, input_b1_rank].values])

array([[ 779,  738, 1336, ..., -862, -235, -865]])

In [37]:
input_array

array([[2406, 2115,  805]])